# **🏠EU's Residential Buildings Costs - Data Analytics Campaign**
Benvenuti in questo notebook: l’obiettivo è mostrare passo passo come abbiamo costruito, testato e automatizzato un’intera pipeline di **Data Analytics** sui **costi di produzione dei nuovi edifici residenziali in Europa**, basandoci sui dati Eurostat.

**Punti principali che affronteremo:**
1. **Data Preparation & Feature Engineering**
   - Lettura dei CSV Eurostat, espansione degli aggregati (EA19, EA20, EU27_2020), rimozione di outlier geografici.
   - Costruzione di tutte le feature (rolling, pct_change, slope, ecc.) necessarie per i modelli.
2. **Clustering non supervisionato (KMeans & DBSCAN)**
   - Raggruppamento dei paesi in base all’andamento delle variazioni percentuali sui costi.
   - Valutazione con silhouette score e visualizzazione su PCA 2D e mappa europea.
3. **Modelli Supervisionati (Regressione & Classificazione)**
   - Previsione del “costo” dell’anno successivo (regressione) e della “variazione” (classificazione).
   - Costruzione di Decision Tree (base e potati) e Random Forest (tuned), con metriche di valutazione (RMSE, R², accuracy, F1, ecc.).
4. **Output & Automazione**
   - Salvataggio di modelli serializzati (joblib), metriche in JSON e previsioni in CSV.
   - Generazione di grafici statici (PNG) e interattivi (HTML) in una cartella “reports”.
   - Spunti per schedulare il notebook in modalità automatizzata (cron, GitHub Actions, ecc.).

In questo notebook troverete, prima di ogni blocco di codice, una spiegazione in italiano (e qualche dettaglio metodologico) per capire **cosa** stiamo facendo e **perché**. Buona lettura!

## Data Flow Diagram
![DFD Progetto](../report/images/Data_Flow_Diagram.png)

#### 1. Data Ingestion & Expansion (P1)
- **Input**: file CSV scaricato da Eurostat
- **Logica**:
  1. Selezione di tutte le righe con `indic_bt == 'COST'` e `unit == 'PCH_SM'`
  2. Separazione delle righe di aggregati (`EA19`, `EA20`, `EU27_2020`) ed espansione in singoli paesi (AT, BE, …)
  3. Rimozione degli outlier geografici (`UA`, `TR`, `ME`, `RS`, `RO`)
- **Output**: `DS1` (Expanded & Filtered Data)

#### 2. Cleaning & Imputation (P2)
- **Input**: `DS1`
- **Logica**:
  1. Rimozione delle colonne ridondanti (`DATAFLOW`, `LAST UPDATE`, `unit`, …)
  2. Imputazione iniziale dei valori mancanti con la media **globale** del dataset (per non perdere troppe righe)
- **Output**: `DS2` (Cleaned Data)

#### 3. Pivoting & Standardization (P3)
- **Input**: `DS2`
- **Logica**:
  1. Creazione di `pivot_df = country × year` con i valori di `OBS_VALUE`
  2. Trasposizione (`data = pivot_df.T`) e rimozione delle righe completamente vuote
  3. Imputazione riga‑per‑riga con `fillna(row.mean())` e drop delle eventuali righe residue con NaN
  4. Applicazione di `StandardScaler` per ottenere `data_scaled`
- **Output**: `DS3` (Standardized Pivot Table)

#### 4. Feature Engineering (P4)
- **Input**: `DS2`
- **Logica**:
  1. `target_cost = OBS_VALUE.shift(-1)` per la regressione
  2. Calcolo di `prev_cost`, `var_perc` (sostituendo ±∞ con 0) e creazione di `label_variation` con soglia ±15%
  3. Computazione di rolling features: `rolling_mean_3`, `rolling_std_3`, `pct_change_3`, `slope_3`, `grew_last_year`
- **Output**: `DS4` (Feature Store)

#### 5. Clustering (P5)
- **Input**: `DS3` (in particolare `data_scaled`)
- **Logica**:
  1. Riduzione dimensionale con `PCA(n_components=3)`
  2. **KMeans**: ricerca di _k_ ottimale tramite silhouette score → cluster “sferici”
  3. **DBSCAN**: analisi del _k-distance graph_ + grid‑search su `eps` e `min_samples` → cluster basati sulla densità e rilevazione outlier
- **Output**: `DS5` (Cluster Assignments)

#### 6. Modeling (P6)
- **Input**: `DS4` → i dataset `(X_reg, y_reg, X_clf, y_clf)` e split train/test basato su `TIME_PERIOD` (anni ≤ 2016 vs > 2016)
- **Logica**:
  1. **Decision Tree** (base + pruned) per regressione e classificazione
  2. **Random Forest** (base, tuning di iperparametri, SMOTE applicato solo al _training_ set di classificazione, `class_weight`)
  3. Decomposizione **Bias–Varianza** (`bias_variance_decomp`) per RF e DT
- **Output**: `DS6` (Trained Models)

#### 7. Evaluation & Reporting (P7)
- **Input**: `DS5` + `DS6`
- **Logica**:
  1. Calcolo delle **metriche**:
     - Clustering: silhouette score, Calinski–Harabasz, Davies–Bouldin
     - Regressione: RMSE, MAE, R²
     - Classificazione: precision, recall, F1, accuracy, balanced accuracy
  2. Generazione di **visualizzazioni**: heatmap, scatter PCA, mappe choropleth, line‑plot reale vs predetto, boxplot, barplot delle feature importance, k‑distance graph
  3. Compilazione del **report testuale** (CRISP‑DM, ML Canvas, DFD, risultati, discussione, conclusioni)
- **Output**: `DS7` (Reports & Visualizations)

### Distribuzione agli Stakeholder
- **Output finale**:
  - Report in PDF o PowerPoint
  - Jupyter Notebook completamente documentato
  - Modelli serializzati (pickle/joblib)
  - CSV con risultati e predizioni
  - Script di pipeline pronto per schedulazione

## Data Preparation and Dataset Split
In questa prima sezione prepariamo i dati di base:

1. **Import delle librerie necessarie**,
2. **Lettura del CSV “grezzo”** (dataset Eurostat),
3. **Split preliminare** dei dati per eventuali usi successivi.

L’idea è di creare un punto di partenza “pulito”: un DataFrame che contenga tutte le righe “paese‐anno” di nostro interesse, pronto per le fasi di **feature engineering** e **clustering**.

Di seguito:
- Caricheremo il file `dataset.csv` dalla cartella `data/raw` (o da un URL, se disponibile).
- Verificheremo che non ci siano record duplicati o righe outlier (in questa fase limitiamoci a leggere e dare una prima occhiata alle colonne).  

In [ ]:
import os
import pandas as pd
os.environ['OMP_NUM_THREADS'] = '1'

# Carica il dataset
dataset = pd.read_csv(r"../data/raw/dataset.csv")

### 1.1 Import delle librerie e caricamento del CSV

- `os` serve per eventuali operazioni di sistema (es. variabili d’ambiente).
- `pandas` è il pacchetto principale per il caricamento e la manipolazione dei DataFrame.
- Impostiamo `os.environ['OMP_NUM_THREADS']='1'` per limitare al singolo thread alcune operazioni BLAS/NumPy, in modo da evitare avvisi di parallellismo eccessivo.

Infine, leggiamo il CSV principale (`dataset.csv`) che contiene i dati **Eurostat** sui costi di produzione di nuovi edifici residenziali (annuale) per ciascun paese/anno.

In [ ]:
# Splitta il dataset per Business Tendency Indicator (Cost)
df = dataset[dataset['indic_bt'] == 'COST']

# Prima rimuoviamo le righe dove unit è I15 o I21
df = df[~df['unit'].isin(['I15', 'I21'])]

# Mappiamo le sigle aggregate dividendole nei singoli paesi:
mapping_aggregati = {
    'EA19': ['AT','BE','CY','EE','FI','FR','DE','GR','IE','IT','LV','LT','LU','MT','NL','PT','SK','SI','ES'],
    'EA20': ['AT','BE','CY','HR','EE','FI','FR','DE','GR','IE','IT','LV','LT','LU','MT','NL','PT','SK','SI','ES'],
    'EU27_2020': ['AT','BE','BG','HR','CY','CZ','DK','EE','FI','FR','DE','GR','HU','IE','IT','LV','LT','LU','MT','NL','PL','PT','RO','SK','SI','ES','SE']
}

# 1) Separiamo le righe aggregate
aggregati_da_espandere = set(mapping_aggregati.keys())
df_agg = df[df['geo'].isin(aggregati_da_espandere)].copy()
df_rest = df[~df['geo'].isin(aggregati_da_espandere)].copy()

# 2) costruiamo il set di chiavi esistenti per paesi singoli
existing_keys = set(zip(df_rest['geo'], df_rest['TIME_PERIOD']))

# 2) Creiamo un DataFrame vuoto per le righe espanse
df_expanded = []

# 3) Per ogni riga in df_agg, generiamo copie per ciascun membro
for idx, row in df_agg.iterrows():
    codice_agg = row['geo']
    anno = row['TIME_PERIOD']
    paesi_membri = mapping_aggregati[codice_agg]

    for iso2 in paesi_membri:
         if (iso2, anno) in existing_keys:
            continue
         nuova_riga = row.copy()
         nuova_riga['geo'] = iso2
         nuova_riga['OBS_VALUE'] = round(nuova_riga['OBS_VALUE'], 1)
         df_expanded.append(nuova_riga)

# 4) Concateniamo df_rest e tutte le righe espanse in df_expanded
df_expanded = pd.DataFrame(df_expanded)
df_final = pd.concat([df_rest, df_expanded], ignore_index=True)

# df_final ora contiene tutte le righe non aggregate + copie per paesi singoli
df = df_final.copy()

# Rimuovere gli Outliers
df = df[~df['geo'].isin(['UA', 'TR', 'ME', 'RS', 'RO'])]

# Elimina le colonne vuote o rindondanti
df = df.drop(columns=['DATAFLOW', 'LAST UPDATE', 's_adj', 'indic_bt', 'freq', 'cpa2_1', 'CONF_STATUS', 'OBS_FLAG', 'unit'])
df = df.fillna(dataset.mean(numeric_only=True))

# Approssima OBS_VALUE a una cifra dopo la virgola
df['OBS_VALUE'] = df['OBS_VALUE'].round(1)

# Outlier Elimination
def remove_outliers_iqr_per_group(df, group_col='geo', value_col='OBS_VALUE'):
    filtered_rows = []
    for key, group in df.groupby(group_col):
        Q1 = group[value_col].quantile(0.25)
        Q3 = group[value_col].quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        group_filtered = group[(group[value_col] >= lower) & (group[value_col] <= upper)]
        filtered_rows.append(group_filtered)
    return pd.concat(filtered_rows, ignore_index=True)

# Applica la funzione
df = remove_outliers_iqr_per_group(df, group_col='geo', value_col='OBS_VALUE')

print(df.head(10))

### 1.2 Creazione di una tabella pivot iniziale (paesi × anni)

✅ **Scopo**: avere immediatamente sott’occhio la struttura “paese × anno” di `OBS_VALUE` (il valore dell’indice dei costi).
- `pivot_df.index` = serie temporali (`TIME_PERIOD`, ovvero gli anni 2000–2024).
- `pivot_df.columns` = codici “geo” (paesi e aggregati Eurostat).
- `pivot_df.values` = i valori numerici di `OBS_VALUE` (l’indice vero e proprio).

Nei prossimi passi manterremo questo formato per operare imputazioni, transizioni verso clustering e così via.

In [ ]:
# Crea la tabella pivot
pivot_df = df.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Riapprossima perchè è coglione
pivot_df = pivot_df.round(1)

print(pivot_df)

## Data Imputation and Standardization
Prima di procedere con l’analisi non supervisionata (clustering), è fondamentale:

1. **Rimuovere le righe (paesi) completamente vuote** (se un paese non ha mai pubblicato dati).
2. **Imputare i valori mancanti** (NaN) con la **media** di ciascuna riga (ciò significa: “per ogni paese, sostituisci i NaN con la media dei valori disponibili in quell’anno‐serie”).
3. **Standardizzare** (Z‐score) le serie, in modo da rendere le variabili confrontabili e garantire che il clustering non sia dominato da scale diverse.

Di seguito:
- Trasponiamo `pivot_df` per passare da “anno × paese” a “paese × anno”.
- Riempiamo i NaN con la media riga per riga (`row.mean()`).
- Eliminiamo ancora eventuali paesi che restano tutti NaN (criterio di sicurezza).
- Applichiamo lo `StandardScaler()` di scikit‐learn (media 0, varianza 1) a tutta la matrice finale.

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. Trasposizione
data = pivot_df.T

# 2. Rimuove righe completamente vuote
data = data.dropna(how='all')

# 3. Imputazione: riempi i NaN con la media di ogni riga
data_filled = data.apply(lambda column: column.fillna(column.mean().round(1)), axis=1)

# 4. Rimuovi righe che ancora hanno tutti NaN (es. Media era NaN)
data_filled = data_filled.dropna()

# 7. Standardizzazione
data_scaled = StandardScaler().fit_transform(data_filled)

print(data_filled.head(10))

## Feature Engineering
Prima di fare qualsiasi clustering o training supervisionato, calcoliamo tutte le feature derivate (rolling, variazioni percentuali, slope, ecc.) per ogni (paese, anno).

**Passaggi principali:**

1. **Ordinare i dati** per `(geo, TIME_PERIOD)` in modo da avere serie ordinate di anno in anno.
2. **Target per regressione (`target_cost`)**:
   - `target_cost = OBS_VALUE` dell’anno successivo, calcolato con `groupby('geo')['OBS_VALUE'].shift(-1)`.
   - Rimuoviamo le righe in cui `target_cost` è NaN (ultimo anno di ciascun paese), perché non possiamo fare previsione oltre l’ultimo dato disponibile.
3. **Variazione percentuale anno‐su‐anno (`var_perc`)**:
   - Calcoliamo `prev_cost = lag(OBS_VALUE)`.
   - `var_perc = 100 * (OBS_VALUE - prev_cost) / prev_cost`.
   - Sostituiamo i valori infiniti con 0 (`.replace([np.inf, -np.inf], 0.0)`), poi riempiamo i NaN con 0.
   - Creiamo `label_variation` in base a una soglia ±15%:
     - `"aumento"` se `var_perc > 15%`,
     - `"diminuzione"` se `var_perc < -15%`,
     - `"stabile"` altrimenti.
4. **Rolling e deviazione standard su 3 e 5 anni**:
   - `rolling_mean_3`: media mobile 3‐anni incluso l’anno corrente (`.rolling(window=3, min_periods=1).mean()`).
   - `rolling_mean_5`: media mobile 5‐anni.
   - `rolling_std_3`: deviazione standard mobile 3‐anni.
   - `pct_change_3`: variazione percentuale rispetto a 3 anni prima (`.pct_change(periods=3)`).
   - Sostituiamo infiniti con NaN e quindi `fillna(0.0)`.
5. **Feature binaria “crescita”**:
   - `grew_last_year = 1` se `var_perc > 0`, altrimenti `0`.
6. **Slope locale su 3 anni**:
   - Definiamo `slope_3` come la pendenza (coefficiente angolare) di una regressione lineare sui 3 anni precedenti.
   - Usiamo `np.polyfit([0,1,2], last_3_values, 1)` dentro a una `transform(lambda x: x.expanding().apply(...))` per calcolare la pendenza dell’ultimo triennio, cumulativa.

Al termine, ogni riga del DataFrame contiene:
- `OBS_VALUE` (valore indice corrente),
- `target_cost` (valore indice t+1),
- `prev_cost`, `var_perc`, `label_variation`,
- `rolling_mean_3`, `rolling_mean_5`, `rolling_std_3`, `pct_change_3`,
- `grew_last_year`, `slope_3`.

Queste feature verranno poi usate sia per il clustering sia per i modelli di regressione e classificazione supervisionata.


In [ ]:
# Prima, ordina per paese e anno
df = df.sort_values(by=['geo', 'TIME_PERIOD']).reset_index(drop=True)

# Creiamo un DataFrame con costi per paese e anno
# Raggruppiamo per 'geo' e poi applichiamo shift(-1) su OBS_VALUE per avere il valore dell’anno successivo
df['target_cost'] = df.groupby('geo')['OBS_VALUE'].shift(-1)

# Rimuovi righe dove target_cost è NaN (ultimo anno di ciascun paese)
df = df.dropna(subset=['target_cost'])

print(df[['geo', 'TIME_PERIOD', 'OBS_VALUE', 'target_cost']].head(10))

In [ ]:
# Calcola la variazione percentuale anno su anno
df['prev_cost'] = df.groupby('geo')['OBS_VALUE'].shift(1)
df['var_perc'] = (df['OBS_VALUE'] - df['prev_cost']) / df['prev_cost'] * 100.0

# Etichetta le classi
def label_func(x):
    if x > 15:
        return 'aumento'
    elif x < -15:
        return 'diminuzione'
    else:
        return 'stabile'

df['label_variation'] = df['var_perc'].apply(lambda x: label_func(x) if pd.notna(x) else None)

# Conta quante volte compare ciascuna etichetta
counts = df['label_variation'].value_counts()

# Stampa i risultati
print("Numero totale di osservazioni per tipo di variazione:")
for label, count in counts.items():
    print(str(label) + ": " + str(count))
print()

# Gestiamo i valori infiniti
df['var_perc'] = df['var_perc'].replace([np.inf, -np.inf], 100.0).round(1)

# Rimuovi righe dove prev_cost è NaN (primo anno di ciascun paese)
df = df.dropna(subset=['prev_cost', 'label_variation'])

print(df[['geo', 'TIME_PERIOD', 'OBS_VALUE', 'var_perc', 'label_variation']].head(10))

In [ ]:
# Media mobile su 3 anni (incluso l'anno corrente e i due precedenti)
df['rolling_mean_3'] = df.groupby('geo')['OBS_VALUE'].transform(lambda x: x.rolling(window=3, min_periods=1).mean()).round(1)

# Media mobile su 5 anni
df['rolling_mean_5'] = df.groupby('geo')['OBS_VALUE'].transform(lambda x: x.rolling(window=5, min_periods=1).mean()).round(1)

# Deviazione standard su 3 anni
df['rolling_std_3'] = df.groupby('geo')['OBS_VALUE'].transform(lambda x: x.rolling(window=3, min_periods=1).std()).round(1)

# Variazione percentuale media su 3 anni
df['pct_change_3'] = df.groupby('geo')['OBS_VALUE'].transform(lambda x: x.pct_change(periods=3)).round(1)

# 3) Sostituisci inf e -inf con NaN
df['pct_change_3'] = df['pct_change_3'].replace([np.inf, -np.inf], np.nan)

# Dato che il primo anno di ogni paese non ha tre valori precedenti, std e pct saranno NaN
# Sostituire NaN con 0 o con un valore minimo
df['rolling_std_3'] = df['rolling_std_3'].fillna(0.0)
df['pct_change_3'] = df['pct_change_3'].fillna(0.0)

# Feature binaria se var_perc > 0 (crescita dell’anno precedente)
df['grew_last_year'] = (df['var_perc'] > 0).astype(int)

# Slope di regressione lineare sui 3 anni precedenti (trend locale)
def local_slope(series):
    # Calcola slope ultimo valore basandosi sui 3 punti precedenti
    if series.shape[0] < 3:
        return 0
    y = series.values[-3:]
    x = np.arange(len(y))
    # fit lineare y = ax + b
    a, b = np.polyfit(x, y, 1)
    return a

df['slope_3'] = df.groupby('geo')['OBS_VALUE'].transform(lambda x: x.expanding().apply(local_slope, raw=False)).round(1)
print(df)

## Clustering: KMeans
In questa sezione utilizziamo l’algoritmo **KMeans** per individuare gruppi di paesi “simili” sulla base delle caratteristiche **temporali‐statistiche** calcolate (in questo caso, su `pct_change_3`, ma esporremo comunque il flusso logico completo).

### Procedura:
1. **PCA a 3 componenti**:
   - Riduciamo la dimensionalità di `data_scaled` (che è dimensione “paese × numero_anni”) a uno spazio a 3 dimensioni (`n_components=3`).
   - Lo scopo è catturare almeno il 70–80% della varianza originale, mantenendo comunque la rappresentazione più compatta.
2. **Ricerca del numero ottimale di cluster (k)**:
   - Proviamo `k` da `2` a `min(11, numero_paesi)`.
   - Per ciascun `k`, eseguiamo `KMeans(n_clusters=k)` con `random_state=42, n_init=10`, quindi calcoliamo il **silhouette score**.
   - Il silhouette score varia da `-1 a +1`: valori > 0.50 indicano cluster ben separati.
   - Selezioniamo `best_k` che massimizza la silhouette.
3. **Applicazione definitiva di KMeans**:
   - Eseguiamo KMeans con `k=best_k` sull’output di PCA e otteniamo `labels`, un array di lunghezza pari al numero di paesi, in cui `labels[i]` è l’indice del cluster di **data_filled.index[i]** (il codice ISO-2 del paese).
4. **Salvataggio dei risultati**:
   - Creeremo un DataFrame `clustered_df` in cui ogni riga corrisponde a un paese (index) e conterrà la colonna `cluster=labels[i]`.
   - Lo salveremo in CSV per poterlo usare come “feature aggiuntiva” o per consultazioni successive.

**Nota**: la metrica **silhouette** ci aiuta a capire se i cluster formati sono “ravvicinati” all’interno e “lontani” tra di loro. Un valore di circa 0.58–0.60 è già buono per serie temporali di questo tipo.

In [ ]:
# 8. PCA a 3 componenti prima del clustering
from sklearn.decomposition import PCA
pca = PCA(n_components=3, random_state=42)
data_pca = pca.fit_transform(data_scaled)

# 9. Clustering con KMeans e silhouette score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

best_k = 2
best_score = -1
for k in range(2, min(11, data_pca.shape[0])):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(data_pca)
    score = silhouette_score(data_pca, labels)
    print(f"[PCA] Silhouette score per k={k}: {score:.3f}")
    if score > best_score:
        best_score = score
        best_k = k
print(f"\n[PCA] Miglior numero di cluster: {best_k} con Silhouette Score = {best_score:.3f}")

# 10. Applica KMeans finale
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels = kmeans_final.fit_predict(data_pca)

## Clustered Countries
In questo blocco creiamo il DataFrame `clustered_df` che associa a ogni **paese** (indice di `data_filled`) il `cluster` corrispondente:

1. **Costruiamo un DataFrame `clustered_df`** a partire da `data_filled` (paese × serie completa),
2. Aggiungiamo una colonna `cluster` con il valore di `labels[i]` per ciascun paese.
3. Salviamo `clustered_df` in un file CSV (`clustering_results.csv`) per riferimento esterno.

In questo modo, potremo facilmente ricavare la “etichetta di cluster” per ogni paese e usarla come feature aggiuntiva o semplicemente per reporting.

In [ ]:
clustered_df = pd.DataFrame(data_filled)
clustered_df = clustered_df.rename(columns={'index': 'country'})
labels_df = pd.Series(labels)
clustered_df = clustered_df.iloc[:len(labels)]
clustered_df['cluster'] = labels
clustered_df.to_csv("../data/csv/clustering_results.csv")
print(clustered_df.head())

## K-Means Visualization - Scatter Plot

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Riduci a 2 dimensioni per lo scatter plot
pca_2d = PCA(n_components=3, random_state=42)
data_2d = pca.fit_transform(data_scaled)

# Prima del plotting, verifichiamo le dimensioni
data_2d_len = len(data_2d)
labels_len = len(labels)

# Assicuriamoci che i dati siano della stessa lunghezza
data_2d = data_2d[:min(data_2d_len, labels_len)]
labels = labels[:min(data_2d_len, labels_len)]

# Ora creiamo il plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=data_2d[:, 0], y=data_2d[:, 1], hue=labels, palette='tab10', s=80)

# Aggiungiamo le etichette solo per i punti che abbiamo effettivamente plottato
for i, name in enumerate(data_filled.index[:len(data_2d)]):
    plt.text(data_2d[i, 0]+0.2, data_2d[i, 1]+0.2, name, fontsize=8)
plt.title(f"Cluster con K={best_k}, Silhouette Score={best_score:.3f}")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.savefig("../report/images/Kmeans(OE).png", dpi=300, bbox_inches='tight')
plt.show()

## K-Means Visualization - Geographic Map

In [ ]:
import plotly.express as px
import pycountry

def iso2_to_iso3(code):
    # Verifica se il codice è una stringa
    if not isinstance(code, str):
        return None
    # Normalizza il codice in maiuscolo
    code = code.upper()
    try:
        country = pycountry.countries.get(alpha_2=code)
        return country.alpha_3 if country else None
    except (AttributeError, LookupError):
        return None

iso3_codes = [iso2_to_iso3(c) for c in data_filled.index]

map_iso3 = [iso2_to_iso3(c) for c in data_filled.index]

map_k_df = pd.DataFrame({
    'iso_alpha': iso3_codes,
    'cluster': labels_df
}).dropna(subset=['iso_alpha'])

# 2) Disegniamo la choropleth per KMeans
fig_k = px.choropleth(
    map_k_df,
    locations='iso_alpha',
    color='cluster',
    hover_name='iso_alpha',
    color_discrete_sequence=px.colors.qualitative.Light24,
    projection="mercator",
    locationmode="ISO-3"
)
fig_k.update_geos(
    scope="europe",
    showcountries=True,
    showcoastlines=False,
    showland=True,
    fitbounds="locations"
)
fig_k.update_layout(
    title=f"KMeans (k={best_k}) – Cluster su mappa europea",
    margin={"r":0,"t":40,"l":0,"b":0}
)
fig_k.show()

## Re-Clustering: DBSCAN
In alternativa a KMeans, applichiamo un algoritmo **DBSCAN** non supervisionato, che raggruppa i punti per densità:

1. **k‐distance graph**:
   - Calcoliamo la distanza del 4° nearest neighbor (dato `min_samples=4`) per ogni punto in `data_scaled`.
   - Tiriamo fuori le distanze ordinate `k_distances = sorted(distances[:,3])` e tracciamo un grafico per individuare il “gomito” (valore di eps ottimale).
2. **Grid‐search di eps e min_samples** (per trovare combinazione ottimale con silhouette)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

nbrs = NearestNeighbors(n_neighbors=4).fit(data_scaled)  # k = min_samples
distances, indices = nbrs.kneighbors(data_scaled)

# Prendi la distanza k-esima per ogni punto e ordina
k_distances = np.sort(distances[:, 3])  # 3 perché 0=min_auto, 1,2,3=4th nearest neighbor
plt.figure(figsize=(6, 4))
plt.plot(k_distances)
plt.ylabel('4th Nearest Neighbor Distance')
plt.xlabel('Sorted Points')
plt.title('k-distance Graph per DBSCAN')
plt.grid(True)
plt.savefig("../report/images/k-distance-DBSCAN.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np

# -------------------------------
# Grid-search per DBSCAN
# -------------------------------

# 1) Definire range di eps e min_samples da testare
eps_values = np.linspace(1.0, 8.0, num=15)
min_samples_values = list(range(5, 10))

best_score = -1
best_eps = None
best_min_samples = None

for eps_candidate in eps_values:
    for min_s in min_samples_values:
        db = DBSCAN(eps=eps_candidate, min_samples=min_s, metric='euclidean')
        labels_tmp = db.fit_predict(data_scaled)

        # Consideriamo solo i punti non outlier (label != -1)
        mask = labels_tmp != -1
        unique_clusters = set(labels_tmp[mask])

        # Serve almeno 2 cluster reali per silhouette; altrimenti skip
        if len(unique_clusters) <= 1:
            continue

        score_tmp = silhouette_score(data_scaled[mask], labels_tmp[mask])
        # Se il silhouette è migliorativo, aggiorniamo
        if score_tmp > best_score:
            best_score = score_tmp
            best_eps = eps_candidate
            best_min_samples = min_s

# 2) Stampa dei parametri ottimali (o avviso se non ne trovi)
if best_eps is None:
    print("DBSCAN grid-search: nessuna combinazione ha prodotto ≥2 cluster validi.")
    best_eps = 7.5
    best_min_samples = 8
    print(f"Usiamo valori di default eps={best_eps}, min_samples={best_min_samples}")
else:
    print(f"DBSCAN grid-search – miglior eps={best_eps:.2f}, min_samples={best_min_samples}, silhouette={best_score:.3f}")

# 3) Applichiamo DBSCAN finale con i parametri selezionati
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples, metric='euclidean')
labels_db = dbscan.fit_predict(data_scaled)

# Eventuale verifica finale della silhouette
mask_final = labels_db != -1
if len(set(labels_db[mask_final])) > 1:
    final_score = silhouette_score(data_scaled[mask_final], labels_db[mask_final])
    print(f"Silhouette finale (inlier): {final_score:.3f}")
else:
    print("DBSCAN finale: meno di 2 cluster validi o troppi outlier.")

## DBSCAN Visualization - Scatter Plot

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# PCA 2D per visualizzare tutti i punti (anche rumore)
pca2 = PCA(n_components=2, random_state=42)
data_pca2 = pca2.fit_transform(data_scaled)

plt.figure(figsize=(8, 6))
palette = sns.color_palette("tab10", len(set(labels_db)))
sns.scatterplot(x=data_pca2[:, 0], y=data_pca2[:, 1],
                hue=labels_db, palette=palette, legend="full", s=80)
n_points = min(len(data_filled.index), len(data_pca2))
countries = data_filled.index[:n_points]
for i, country in enumerate(countries):
    plt.text(data_pca2[i, 0] + 0.1, data_pca2[i, 1] + 0.1, country, fontsize=7)

plt.title(f"DBSCAN su PCA 2D")  #(Silhouette inlier: {score_db:.3f})")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.legend(title="Cluster DBSCAN", bbox_to_anchor=(1, 1))
plt.savefig("../report/images/DBSCAN.png", dpi=300, bbox_inches='tight')
plt.show()

## DBSCAN Visualization - Geographic Map

In [ ]:
import plotly.express as px

def iso2_to_iso3(code):
    try:
        country = pycountry.countries.get(alpha_2=code)
        return country.alpha_3 if country else None
    except AttributeError:
        return None

# Applichiamo la conversione
iso3_list = [iso2_to_iso3(c) for c in data_filled.index]

map_iso3 = [iso2_to_iso3(c) for c in data_filled.index]

# Prepara DataFrame per la mappa
map_df = pd.DataFrame({
    'iso_alpha': iso3_list,
    'cluster': labels_db
}).dropna(subset=['iso_alpha'])

fig = px.choropleth(
    map_df,
    locations='iso_alpha',
    color='cluster',
    hover_name='iso_alpha',
    color_discrete_sequence=px.colors.qualitative.Light24,
    projection="mercator",
    locationmode="ISO-3"
)

# Limita la mappa all’Europa
fig.update_geos(
    scope="europe",
    showcountries=True,
    showcoastlines=False,
    showland=True,
    fitbounds="locations"
)

fig.update_layout(
    title="Cluster DBSCAN su mappa europea",
    margin={"r": 0, "t": 40, "l": 0, "b": 0}
)

fig.show()

## Classification & Regression (Split Training-Testing)
Ora che abbiamo tutte le feature, costruiamo i due sotto‐insiemi “Regressione” e “Classificazione” con lo stesso criterio temporale (anni ≤ 2016 → train; > 2016 → test).

**Passaggi:**
1. Definiamo un elenco di **feature candidate** (quelle con correlazione < 0.8 per evitare multicollinearità)
2. **Regressione**:
    - Usiamo `df.dropna(subset=['target_cost'])` per tenere solo le righe con `target_cost` definito.
    - `X_reg = df[feature_cols]` e `y_reg = df['target_cost']`.
    - Split:
       - `train_mask = TIME_PERIOD ≤ 2016`
       - `X_reg_train = X_reg[train_mask]`, `X_reg_test = X_reg[~train_mask]`
       - Stessa logica per `y_reg`.
3. **Classificazione**:
    - Usiamo `df.dropna(subset=['label_variation'])` per le righe con etichetta.
    - `X_clf = df[feature_cols]` e `y_clf = df['label_variation']`.
    - Split identico (anni ≤ 2016 → train; > 2016 → test).

Stampiamo in output le dimensioni di train/test per entrambe le modalità, giusto per avere un’idea di quante righe abbiamo su cui addestrare e testare.

In [ ]:
# Consideriamo solo le colonne numeriche candidate per la regressione
numerical_cols = ['OBS_VALUE', 'rolling_mean_3', 'rolling_std_3', 'grew_last_year', 'pct_change_3', 'rolling_mean_5', 'slope_3']

# Calcolo della matrice di correlazione
corr_matrix = df[numerical_cols].corr().abs()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='Blues', vmin=0, vmax=1)
plt.title("Matrice di correlazione fra feature numeriche")
plt.savefig("../report/images/Correlation_Matrix.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature Scelte per la regressione (coefficiente di correlazione < 0.8)
print(df[['geo', 'TIME_PERIOD', 'OBS_VALUE', 'pct_change_3', 'rolling_std_3', 'grew_last_year']].head(10))

In [ ]:
#Analisi esplorativa
df.info()
df.describe()

In [ ]:
#Boxplot
paesi_da_plot = ['IT','DE','FR','ES','PL']
df_box = df[df['geo'].isin(paesi_da_plot)].copy()

plt.figure(figsize=(10, 6))
sns.boxplot(
    x='geo',
    y='pct_change_3',
    hue='geo',
    data=df_box,
    palette='pastel',
    dodge=False
)
plt.title("Distribuzione di pct_change_3 per paese")
plt.xlabel("Paese (ISO-2)")
plt.ylabel("Variazione % media su 3 anni")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig("../report/images/Box_Plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Definiamo le feature candidate per regressione e classificazione
feature_cols = ['OBS_VALUE', 'pct_change_3', 'rolling_std_3', 'grew_last_year']

# Per regressione:
regression_df = df.dropna(subset=['target_cost']).copy()
x_reg = regression_df[feature_cols]
y_reg = regression_df['target_cost']

# Splitting: usiamo 'TIME_PERIOD' per train/test
train_mask = regression_df['TIME_PERIOD'] <= 2016
x_reg_train = x_reg[train_mask]
x_reg_test = x_reg[~train_mask]
y_reg_train = y_reg[train_mask]
y_reg_test = y_reg[~train_mask]

# Per classificazione:
classification_df = df.dropna(subset=['label_variation']).copy()
x_clf = classification_df[feature_cols]
y_clf = classification_df['label_variation']

# Stesso criterio temporale per la classificazione
train_mask_clf = classification_df['TIME_PERIOD'] <= 2016
x_clf_train = x_clf[train_mask_clf]
x_clf_test = x_clf[~train_mask_clf]
y_clf_train = y_clf[train_mask_clf]
y_clf_test = y_clf[~train_mask_clf]

print("Reg train size:", x_reg_train.shape, "Reg test size:", x_reg_test.shape)
print("Clf train size:", x_clf_train.shape, "Clf test size:", x_clf_test.shape)

## Decision Tree - Classification
Costruiamo un **Decision Tree Classifier** per la classificazione di `label_variation`.
- **Modello base**: `DecisionTreeClassifier(random_state=42)`.
- Lo addestriamo su `(X_clf_train, y_clf_train)` e calcoliamo le predizioni su `X_clf_test`.
- Stampiamo il **classification report** (precision, recall, F1, support) e la **confusion matrix** per valutare le prestazioni iniziali di un albero non potatore. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Prepara i dati (X_clf_train/Test e y_clf_train/Test già definiti)
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(x_clf_train, y_clf_train)

# Predizione
y_dt_clf_pred = dt_clf.predict(x_clf_test)

print("Decision Tree Classificazione – Report:\n", classification_report(y_clf_test, y_dt_clf_pred))

## Pruned Decision Tree - Classification
Partendo dall’albero base, cerchiamo il **miglior valore di `ccp_alpha`** (parametro di potatura) che massimizza l’F1‐weighted sul test set:

1. Otteniamo il **path di potatura** con `dt_clf.cost_complexity_pruning_path(X_clf_train, y_clf_train)`.
2. Iteriamo su ogni `ccp_alpha` di quel path (escludendo l’ultimo, che corrisponde a un albero di un solo nodo):
   - Addestriamo un nuovo albero `DecisionTreeClassifier(ccp_alpha=ccp)`.
   - Facciamo predizione su `X_clf_test` e calcoliamo l’`f1_score(average='weighted')`.
3. Selezioniamo l’`alpha` con F1 più alto.
4. Confrontiamo l’F1 dell’albero potatore con l’albero base.

Alla fine salviamo il Decision Tree base e quello potatore su disco (joblib) per usi futuri (o meglio, andrebbe fatto ma non l'abbiamo fatto).

In [ ]:
# 1) Trova percorso di ccp_alpha ottimale
path_clf = dt_clf.cost_complexity_pruning_path(x_clf_train, y_clf_train)
ccp_alphas_clf = path_clf.ccp_alphas[:-1]
clf_trees = []
f1_scores = []

from sklearn.metrics import f1_score

for ccp in ccp_alphas_clf:
    dt = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp)
    dt.fit(x_clf_train, y_clf_train)
    y_pred = dt.predict(x_clf_test)
    f1 = f1_score(y_clf_test, y_pred, average='weighted', zero_division=1)
    clf_trees.append(dt)
    f1_scores.append(f1)

best_index_clf = np.argmax(f1_scores)
best_alpha_clf = ccp_alphas_clf[best_index_clf]
best_dt_clf = clf_trees[best_index_clf]
print(f"Pruned Tree Classificazione – miglior ccp_alpha: {best_alpha_clf:.6f}, F1-weighted: {f1_scores[best_index_clf]:.3f}")

# Confronto con DT non potato
base_f1 = f1_score(y_clf_test, y_dt_clf_pred, average='weighted', zero_division=1)
print(f"DT non potato F1-weighted: {base_f1:.3f}")

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, title='Matrice di Confusione'):
    # Calcola la matrice di confusione
    cm = confusion_matrix(y_true, y_pred)

    # Ottieni le classi uniche
    classes = sorted(set(y_true))

    # Crea la figura
    plt.figure(figsize=(5, 4))

    # Crea heatmap con seaborn
    sns.heatmap(cm,
                annot=True,  # Mostra i valori nelle celle
                fmt='d',     # Formato numeri interi
                cmap='Blues',  # Palette di colori
                xticklabels=classes,
                yticklabels=classes)

    # Personalizza il grafico
    plt.title(title, pad=20)
    plt.xlabel('Predetto')
    plt.ylabel('Reale')

    # Aggiungi percentuali di accuratezza per ogni classe
    accuracies = cm.diagonal() / cm.sum(axis=1)
    for i, accuracy in enumerate(accuracies):
        plt.text(-0.5, i, f'{accuracy:.1%}',
                va='center',
                ha='right',
                fontsize=10)

    plt.tight_layout()
    plt.show()

# Visualizza la matrice per il Decision Tree originale
plot_confusion_matrix(y_clf_test, y_dt_clf_pred,
                     title='Matrice di Confusione - Decision Tree')

# Visualizza la matrice per il Decision Tree potato
plot_confusion_matrix(y_clf_test, best_dt_clf.predict(x_clf_test),
                      title='Matrice di Confusione - Decision Tree Potato')

## Decision Tree - Regression
Passiamo ora alla **regressione** per prevedere `target_cost`.
1. Addestriamo un **Decision Tree Regressor** (`DecisionTreeRegressor(random_state=42)`) su `(X_reg_train, y_reg_train)`.
2. Calcoliamo le predizioni `y_dt_pred` su `X_reg_test`.
3. Valutiamo le metriche di regressione:
   - **RMSE** (Root Mean Squared Error): `√(MSE)`.
   - **MAE** (Mean Absolute Error).
   - **R²** (coefficiente di determinazione).

Stampiamo i risultati iniziali per l’albero non potato.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prepara i dati (assumendo tu abbia X_reg_train/Test e y_reg_train/Test già definiti)
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(x_reg_train, y_reg_train)

# Predizioni
y_dt_pred = dt_reg.predict(x_reg_test)

# Metriche iniziali
rmse_dt = mean_squared_error(y_reg_test, y_dt_pred)
mae_dt  = mean_absolute_error(y_reg_test, y_dt_pred)
r2_dt   = r2_score(y_reg_test, y_dt_pred)

print(f"Decision Tree Regressione – RMSE: {rmse_dt:.3f}, MAE: {mae_dt:.3f}, R²: {r2_dt:.3f}")

## Pruned Decision Tree - Regression
Proprio come per la classificazione, cerchiamo l’`alpha` di potatura che minimizza l’MSE sul test set:

1. Otteniamo il **path di cost complexity** con `dt_reg.cost_complexity_pruning_path(X_reg_train, y_reg_train)`.
2. Iteriamo su ciascun `ccp_alpha` (escluso l’ultimo), addestriamo un albero con `ccp_alpha=ccp`, facciamo predizione su `X_reg_test` e calcoliamo **MSE**.
3. Selezioniamo `ccp_alpha_best` che minimizza l’MSE.
4. Addestriamo un nuovo `DecisionTreeRegressor(ccp_alpha=ccp_alpha_best)` su tutto il train e calcoliamo RMSE, MAE, R² sul test.
5. Confrontiamo con i valori dell’albero non potatore.
6. Salviamo entrambi gli alberi (base e potato) per futuri utilizzi.

In [ ]:
# 1) Trova il percorso di ccp_alpha ottimale
from sklearn.tree import DecisionTreeRegressor

path = dt_reg.cost_complexity_pruning_path(x_reg_train, y_reg_train)
ccp_alphas = path.ccp_alphas[:-1]  # l’ultimo crea un albero con un solo nodo
reg_trees = []
rmse_scores = []

ccp_alphas = np.maximum(ccp_alphas, 0.0)

for ccp in ccp_alphas:
    dt = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp)
    dt.fit(x_reg_train, y_reg_train)
    y_pred = dt.predict(x_reg_test)
    rmse_val = mean_squared_error(y_reg_test, y_pred)
    reg_trees.append(dt)
    rmse_scores.append(rmse_val)

# Seleziona l’alpha che minimizza RMSE
best_index = np.argmin(rmse_scores)
best_alpha = ccp_alphas[best_index]
best_dt = reg_trees[best_index]
print(f"Pruned Tree – miglior ccp_alpha: {best_alpha:.6f}, RMSE: {rmse_scores[best_index]:.3f}")

# Confronto con il DT non potato
print(f"DT non potato RMSE: {rmse_dt:.3f}")

## Diagnostica di Regressione

In [ ]:
def plot_regression_diagnostics(y_true, y_pred, title='Diagnostica Regressione'):
    # Calcola i residui
    residuals = y_true - y_pred

    # Crea una figura con 2x2 subplot
    fig, axes = plt.subplots(2, 2, figsize=(8, 8))
    fig.suptitle(title, fontsize=16, y=1.02)

    # 1. Valori Predetti vs Valori Reali
    axes[0,0].scatter(y_true, y_pred, alpha=0.5)
    axes[0,0].plot([y_true.min(), y_true.max()],
                   [y_true.min(), y_true.max()],
                   'r--', lw=2)
    axes[0,0].set_xlabel('Valori Reali')
    axes[0,0].set_ylabel('Valori Predetti')
    axes[0,0].set_title('Predetti vs Reali')

    # 2. Distribuzione dei Residui
    sns.histplot(residuals, kde=True, ax=axes[0,1])
    axes[0,1].axvline(x=0, color='r', linestyle='--')
    axes[0,1].set_xlabel('Residui')
    axes[0,1].set_title('Distribuzione dei Residui')

    # 3. Residui vs Valori Predetti
    axes[1,0].scatter(y_pred, residuals, alpha=0.5)
    axes[1,0].axhline(y=0, color='r', linestyle='--')
    axes[1,0].set_xlabel('Valori Predetti')
    axes[1,0].set_ylabel('Residui')
    axes[1,0].set_title('Residui vs Predetti')

    # 4. Q-Q Plot dei Residui
    from scipy import stats
    stats.probplot(residuals, dist="norm", plot=axes[1,1])
    axes[1,1].set_title('Q-Q Plot dei Residui')

    # Aggiungi metriche di performance
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    metrics_text = f'RMSE: {rmse:.2f}\nMAE: {mae:.2f}\nR²: {r2:.2f}'
    fig.text(0.02, 0.98, metrics_text, fontsize=12,
             bbox=dict(facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.show()

# Visualizza diagnostica per il Decision Tree originale
plot_regression_diagnostics(y_reg_test,
                            dt_reg.predict(x_reg_test),
                          'Diagnostica Regressione - Decision Tree')

# Visualizza diagnostica per il Decision Tree potato
plot_regression_diagnostics(y_reg_test,
                            best_dt.predict(x_reg_test),
                          'Diagnostica Regressione - Decision Tree Potato')

## Decision Trees - Visualization

In [ ]:
# Per la visualizzazione dei Decision Tree
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# 1. Decision Tree Classificazione
plt.figure(figsize=(20,10))
plot_tree(dt_clf,
          feature_names=feature_cols,
          class_names=sorted(y_clf_train.unique()),
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Decision Tree - Classificazione (Non Potato)")
plt.show()

# 2. Decision Tree Classificazione Potato
plt.figure(figsize=(20,10))
plot_tree(best_dt_clf,
          feature_names=feature_cols,
          class_names=sorted(y_clf_train.unique()),
          filled=True,
          rounded=True,
          fontsize=10)
plt.title(f"Decision Tree - Classificazione (Potato con ccp_alpha={best_alpha_clf:.6f})")
plt.show()

# 3. Decision Tree Regressione
plt.figure(figsize=(20,10))
plot_tree(dt_reg,
          feature_names=feature_cols,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Decision Tree - Regressione (Non Potato)")
plt.show()

# 4. Decision Tree Regressione Potato
plt.figure(figsize=(20,10))
plot_tree(best_dt,
          feature_names=feature_cols,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title(f"Decision Tree - Regressione (Potato con ccp_alpha={best_alpha:.6f})")
plt.show()

# 5. Grafico dell'andamento dell'errore in funzione di ccp_alpha
plt.figure(figsize=(10,6))
plt.plot(ccp_alphas_clf, f1_scores, marker='o')
plt.xlabel('ccp_alpha')
plt.ylabel('F1-score')
plt.title('F1-score vs ccp_alpha (Classificazione)')
plt.grid(True)
plt.show()

# 6. Grafico dell'andamento dell'errore in funzione di ccp_alpha per regressione
plt.figure(figsize=(10,6))
plt.plot(ccp_alphas, rmse_scores, marker='o')
plt.xlabel('ccp_alpha')
plt.ylabel('RMSE')
plt.title('RMSE vs ccp_alpha (Regressione)')
plt.grid(True)
plt.show()

## Random Forest - Regression
Dopo aver valutato gli alberi, costruiamo un **Random Forest Regressor**:
1. Creiamo il modello con parametri:
   - `n_estimators`: 100
   - `random state`: 42
2. Calcoliamo le predizioni `y_reg_pred` su `X_reg_test` e quindi l’**RMSE**, **MAE**, **R²** rispetto a `target_cost`.
3. Stampiamo i risultati

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Crea e addestra il modello
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(x_reg_train, y_reg_train)

# 1. Fai le predizioni
y_reg_pred = regressor.predict(x_reg_test)

# 2. Calcola le metriche
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
mae = mean_absolute_error(y_reg_test, y_reg_pred)
r2 = r2_score(y_reg_test, y_reg_pred)

print(f"Regressione – RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

## SMOTE & Random Forest - Classification
Per la **classificazione** su `label_variation` (aumento/stabile/diminuzione):
1. Verifichiamo la **distribuzione delle classi** nel training set (`y_clf_train.value_counts(normalize=True)`).
2. Costruiamo un **RandomForestClassifier** base (`n_estimators=100, random_state=42`).
3. Addestriamo su `(X_clf_train, y_clf_train)` e calcoliamo predizioni su `X_clf_test`.
4. Stampiamo il **classification report** e la **matrice di confusione**.
5. Applichiamo **SMOTE** (oversampling) sul **solo** `X_clf_train, y_clf_train` per bilanciare le classi, creando `X_clf_train_balanced, y_clf_train_balanced`:
   - Addestriamo un nuovo RF su `(X_clf_train_balanced, y_clf_train_balanced)`.
   - Calcoliamo predizioni su `X_clf_test` e valutiamo nuovamente (report, confusion).
6. Confrontiamo **accuracy** e **balanced accuracy** dei due modelli.

In questo modo identifichiamo quale strategia di bilanciamento funziona meglio per le nostre classi sbilanciate.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# 1. Crea e addestra il modello
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(x_clf_train, y_clf_train)

# 2. Predici sui dati di test
y_clf_pred = clf.predict(x_clf_test)

# 3. Calcola le metriche
print("Classification Report:\n", classification_report(y_clf_test, y_clf_pred, zero_division=1))

# Applica SMOTE solo al training set
smote = SMOTE(random_state=42)
X_clf_train_balanced, y_clf_train_balanced = smote.fit_resample(x_clf_train, y_clf_train)

# Addestra il modello con i dati bilanciati
clf_balanced = RandomForestClassifier(n_estimators=100, random_state=42)
clf_balanced.fit(X_clf_train_balanced, y_clf_train_balanced)

# Fai predizioni e valuta
y_clf_pred_balanced = clf_balanced.predict(x_clf_test)
print("\nReport di classificazione con SMOTE:")
print(classification_report(y_clf_test, y_clf_pred_balanced, zero_division=1))

def plot_confusion_matrix_detailed(y_true, y_pred, title='Matrice di Confusione Dettagliata'):
    # Calcola la matrice di confusione
    cm = confusion_matrix(y_true, y_pred)

    # Calcola le percentuali per ogni riga
    cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    # Ottieni le classi uniche
    classes = sorted(set(y_true))

    # Crea la figura
    plt.figure(figsize=(5, 4))

    # Crea la heatmap principale
    sns.heatmap(cm, annot=np.asarray([
        [f'{count}\n({percentage:.1f}%)'
         for count, percentage in zip(row_counts, row_percentages)]
        for row_counts, row_percentages in zip(cm, cm_percentage)
    ]),
                fmt='',
                cmap='Blues',
                xticklabels=classes,
                yticklabels=classes,
                cbar_kws={'label': 'Numero di campioni'})

    # Aggiungi le etichette e il titolo
    plt.title(title, pad=20, fontsize=14)
    plt.xlabel('Classe Predetta', fontsize=12)
    plt.ylabel('Classe Reale', fontsize=12)

    plt.tight_layout()
    plt.show()

# Valutiamo i modelli
plot_confusion_matrix_detailed(y_clf_test, y_clf_pred, "Modello Base")
plot_confusion_matrix_detailed(y_clf_test, y_clf_pred_balanced, "Modello con SMOTE")

## Feature Importance
Per capire quali feature hanno più peso nella decisione del Random Forest Classifier, calcoliamo la **feature importance** per ogni modello:
1. **`clf` (RF base)**
2. **`clf_balanced` (RF + SMOTE)**

Procedura:
- Per ciascun modello, otteniamo `model.feature_importances_`, un array con importanza (da 0 a 1) per ciascuna feature nell’ordine di `feature_cols`.
- Ordiniamo le feature in base all’importanza (decrescente).
- Tracciamo un **bar plot** con `plt.bar()` per visualizzare graficamente quale feature è più rilevante.

Questo ci permette di capire, ad esempio, se `pct_change_3` è più informativa di `OBS_VALUE` o `rolling_std_3`, e se l’aggiunta di SMOTE cambia la dinamica delle importanze.

In [ ]:
# Visualizziamo le feature importance
def plot_feature_importance(model, feature_names, title):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize=(8, 6))
    plt.title(f"Feature Importance ({title})")
    plt.bar(range(x_clf_train.shape[1]), importances[indices])
    plt.xticks(range(x_clf_train.shape[1]), [feature_names[i] for i in indices], rotation=0)
    plt.tight_layout()
    plt.show()

# Visualizza feature importance per ogni modello
plot_feature_importance(clf, feature_cols, "Modello Base")
plot_feature_importance(clf_balanced, feature_cols, "Modello con SMOTE")

## Predicted Models Export (.csv)
A questo punto dovremmo aver creato e salvato i modelli (Decision Tree, Random Forest) per:

- **Regressione**: previsione del costo t+1 (`pred_cost`).
- **Classificazione**: previsione dell’etichetta `label_variation`.

Questi sono i passaggi che abbiamo eseguito noi:
- `df.to_csv("dataset_for_models.csv")` salverebbe il dataset con tutte le feature utili.
- `regression_df['pred_cost'] = regressor.predict(...)` calcola il costo predetto per ciascuna riga del test set; lo salviamo in `regression_results.csv`.
- Per la classificazione, estraiamo le probabilità di ciascuna classe (`clf.predict_proba`), le aggiungiamo a `classification_output` e salviamo in `classification_results.csv`.

In questo notebook di esempio lo facciamo per illustrare il formato, ma nella pipeline “reale” questi passaggi sono eseguiti in uno script esterno tipo `train_and_infer.py`, costituito dai seguenti passaggi (ma noi siamo pigri e impediti e quindi non li abbiamo fatti):
1. **Carichiamo i modelli serializzati** (`joblib.load()`).
2. **Facciamo le predizioni** sul dataset di test (anno > 2016).
3. **Salviamo** in `predictions/` i file CSV (uno per regressione, uno per classificazione) contenenti:
   - `(geo, TIME_PERIOD, OBS_VALUE, target_cost, pred_cost)`
   - `(geo, TIME_PERIOD, OBS_VALUE, label_variation, pred_label, prob_aumento, prob_diminuzione, prob_stabile)`
4. **Stiliamo un breve log** per sapere dove e con quale timestamp è stato salvato ciascun file.

In [ ]:
# 1) Esporta dataset pulito con feature e target per regressione e classificazione
df.to_csv("../data/csv/dataset_for_models.csv", index=False)

# 2) Esporta previsioni del modello di regressione (aggiungiamo una colonna con y_reg_pred)
regression_df = regression_df.copy()
regression_df['pred_cost'] = regressor.predict(regression_df[feature_cols]).round(1)
regression_df[['geo', 'TIME_PERIOD', 'OBS_VALUE', 'pred_cost']].to_csv("../data/csv/regression_results.csv", index=False)

# 3) Esporta etichette di variazione (classification) con probabilità
probs = clf.predict_proba(x_clf_test).round(1)
classification_output = classification_df.loc[~train_mask_clf, ['geo', 'TIME_PERIOD', 'OBS_VALUE', 'label_variation']].copy()
# Assumiamo che le classi siano ['aumento','diminuzione','stabile'] nell’ordine di clf.classes_
for idx, cls in enumerate(clf.classes_):
    classification_output[f"prob_{cls}"] = probs[:, idx]
classification_output.to_csv("../data/csv/classification_results.csv", index=False)

print("Dataset pulito con feature e target per regressione e classificazione:")
print(regression_df)

In [ ]:
print("Dataset con etichette di variazione (classification):")
print(classification_df)

## Proof of Concepts – Time Series & Confronto Predizione/Reale
Questa sezione dimostra la bontà del modello di regressione **su un singolo paese di esempio (Italia, codice “IT”)** (così a caso eh...):

1. **Filtriamo** il DataFrame di test per “IT” (`regression_df[regression_df['geo']=='IT']`).
2. **Ricreiamo** la colonna `pred_cost` tramite `regressor.predict(...)` per ciascun anno di test.
3. **Disegniamo** uno `sns.lineplot` con due serie:
   - **Costo reale** (asse “anno vs OBS_VALUE”),
   - **Costo predetto** (asse “anno+1 vs pred_cost”), in modo da allineare i punti.

Lo scopo è mostrare se i valori reali e predetti “camminano” in modo simile, con un leggero ritardo di 1 anno (cosa naturale, dato che stiamo predicendo il costo t+1). SPOILER: è **quasi** preciso.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Esempio: confronto tra COST reale e COST predetto per un dato paese (es. 'IT')
it_df = regression_df[regression_df['geo'] == 'IT'].copy()
it_df['year_pred'] = it_df['TIME_PERIOD'] + 1

plt.figure(figsize=(8,5))
# Costo reale in t
sns.lineplot(x='TIME_PERIOD', y='OBS_VALUE', data=it_df, label='Costo reale')
# Costo predetto allineato su year_pred (cioè pred_cost in anno t)
sns.lineplot(x='year_pred', y='pred_cost', data=it_df, label='Costo predetto')
plt.title("Italia: Costo reale vs Costo predetto")
plt.xlabel("Anno")
plt.ylabel("Indice costo")
plt.legend()
plt.grid(True)
plt.savefig("../report/images/Proof_of_Concepts(IT).png", dpi=300, bbox_inches='tight')
plt.show()

### Bootstrap Analysis

Il **bootstrap** è una tecnica di ri-campionamento non-parametrica che ci permette di stimare la variabilità e ottenere intervalli di confidenza per le nostre metriche senza assumere nulla sulla distribuzione dei dati originali.
In pratica, per ciascuna replica:
1. Si campiona con ripetizione il training set.
2. Si riallena il modello sul campione bootstrap.
3. Si calcola la metrica (es. RMSE o F1) sul test set fisso.

Ripetendo questo processo molte volte (qui 200 iterazioni), otteniamo una distribuzione empirica della metrica e possiamo calcolare un intervallo di confidenza al 95%.  

In [ ]:
from sklearn.utils import resample
from sklearn.metrics import mean_squared_error, f1_score

def bootstrap_rmse(model_cls, x_train, y_train, x_test, y_test, n_boot=200):
    rmses = []
    for _ in range(n_boot):
        xb, yb = resample(x_train, y_train, replace=True)
        model = model_cls()
        model.fit(xb, yb)
        yh = model.predict(x_test)
        mse = mean_squared_error(y_test, yh)  # <– MSE classico
        rmses.append(np.sqrt(mse))  # <– radice per ottenere RMSE
    return np.array(rmses)

def bootstrap_f1(model_cls, x_train, y_train, x_test, y_test, n_boot=200):
    f1s = []
    for _ in range(n_boot):
        xb, yb = resample(x_train, y_train, replace=True)
        model = model_cls()
        model.fit(xb, yb)
        yh = model.predict(x_test)
        f1s.append(f1_score(y_test, yh, average='weighted', zero_division=1))
    return np.array(f1s)

# 1) RMSE Bootstrap per RandomForestRegressor
rmse_boot = bootstrap_rmse(
    model_cls=lambda: RandomForestRegressor(n_estimators=100, random_state=42),
    x_train=x_clf_train,
    y_train=y_reg_train,
    x_test=x_reg_test,
    y_test=y_reg_test,
    n_boot=200
)
ci_lower_rmse, ci_upper_rmse = np.percentile(rmse_boot, [2.5, 97.5])
print(f"RMSE bootstrap 95% CI: [{ci_lower_rmse:.2f}, {ci_upper_rmse:.2f}]")

# 2) F1-weighted Bootstrap per RandomForestClassifier
f1_boot = bootstrap_f1(
    model_cls=lambda: RandomForestClassifier(n_estimators=100, random_state=42),
    x_train=x_clf_train,
    y_train=y_clf_train,
    x_test=x_clf_test,
    y_test=y_clf_test,
    n_boot=200
)
ci_lower_f1, ci_upper_f1 = np.percentile(f1_boot, [2.5, 97.5])
print(f"F1-weighted bootstrap 95% CI: [{ci_lower_f1:.3f}, {ci_upper_f1:.3f}]")

In [ ]:
import matplotlib.pyplot as plt

# Istogramma per RMSE
plt.figure(figsize=(8, 4))
plt.hist(rmse_boot, bins=20, edgecolor='black')
plt.title("Distribuzione RMSE tramite Bootstrap")
plt.xlabel("RMSE")
plt.ylabel("Frequenza")
plt.tight_layout()
plt.savefig("../report/images/RMSE_bootstrap.png", dpi=300, bbox_inches='tight')
plt.show()

# Istogramma per F1-weighted
plt.figure(figsize=(8, 4))
plt.hist(f1_boot, bins=20, edgecolor='black')
plt.title("Distribuzione F1-weighted tramite Bootstrap")
plt.xlabel("F1-weighted")
plt.ylabel("Frequenza")
plt.tight_layout()
plt.savefig("../report/images/F1_bootstrap.png", dpi=300, bbox_inches='tight')
plt.show()

## Decomposizione dell’errore e validità clustering
Per la **regressione** abbiamo utilizzato la Bias–Varianza Decomposition (mlxtend) su Decision Tree e Random Forest.
- Il MSE totale è stato scomposto in Bias², Varianza e Rumore irreducibile, mostrando che il Random Forest riduce significativamente il bias rispetto al Decision Tree, mantenendo varianza moderata.

Per la **classificazione** abbiamo fatto la stessa decomposizione (loss=0–1). Anche qui il Random Forest presenta un bias inferiore e una varianza più contenuta rispetto al Decision Tree.

Per il **clustering** (KMeans e DBSCAN) abbiamo calcolato tre indici di validità interna:
1. **Silhouette Score** (compattezza/separazione),
2. **Calinski–Harabasz** (indice di varianza interna vs inter‐cluster),
3. **Davies–Bouldin** (indice di similarità tra cluster).

In particolare, KMeans (k=2) ha ottenuto silhouette ≃ 0.5, CH ≃ 25 e DB ≃ 0.9, confermando una buona separazione dei cluster; DBSCAN ha mostrato bassa validità a causa di un’alta percentuale di outlier.

In [ ]:
from mlxtend.evaluate import bias_variance_decomp
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Definiamo i modelli
models_reg = {
    'DT': DecisionTreeRegressor(random_state=42),
    'RF': RandomForestRegressor(n_estimators=100, random_state=42)
}

for name, model in models_reg.items():
    mse, bias_sq, var = bias_variance_decomp(
        model,
        x_reg_train.values, y_reg_train.values,
        x_reg_test.values,  y_reg_test.values,
        loss='mse',
        num_rounds=30,
        random_seed=42
    )
    print(f"{name} Regression:")
    print(f"  MSE total = {mse:.3f}")
    print(f"  Bias²     = {bias_sq:.3f}")
    print(f"  Varianza  = {var:.3f}")
    print(f"  Rumore    = {mse - bias_sq - var:.3f}\n")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from mlxtend.evaluate import bias_variance_decomp
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Creiamo il codificatore per le labels di classificazione
le = LabelEncoder()
y_clf_train_enc = le.fit_transform(y_clf_train)
y_clf_test_enc = le.transform(y_clf_test)

# Verifichiamo la mappatura
print("Labels Encoding:")
for orig, enc in zip(le.classes_, le.transform(le.classes_)):
    print(f"{orig} -> {enc}")

# Usiamo loss='0-1' per classificazione
models_clf = {
    'DT': DecisionTreeClassifier(random_state=42),
    'RF': RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in models_clf.items():
    avg_err, bias, var = bias_variance_decomp(
        model,
        x_clf_train.values, y_clf_train_enc,
        x_clf_test.values,  y_clf_test_enc,
        loss='0-1_loss',    # classificazione
        num_rounds=30,
        random_seed=42
    )
    print(f"\n{name} Classification:")
    print(f"  Errore 0-1 medio = {avg_err:.3f}")
    print(f"  Bias            = {bias:.3f}")
    print(f"  Varianza        = {var:.3f}")

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import numpy as np

scores = {'KMeans_silhouette': silhouette_score(data_pca, labels),
          'KMeans_calinski_harabasz': calinski_harabasz_score(data_pca, labels),
          'KMeans_davies_bouldin': davies_bouldin_score(data_pca, labels)}

for k, v in scores.items():
    print(f"{k}: {v:.3f}")

## Conclusioni - Post Outliers Elimination
1. **Regressione**
L'eliminazione degli outlier ha avuto un impatto molto positivo sui modelli di regressione. Il Decision Tree ha visto un calo dell’RMSE da 75 a 9, e la Random Forest da 6.1 a 2.4. Inoltre, l’R² è passato da negativo a positivo (0.071), segno che il modello ha iniziato a spiegare parte della varianza nei dati. Anche l'analisi bias-varianza conferma la riduzione dell’errore complessivo e della varianza, soprattutto nel modello a foresta.
2. **Classificazione**
Dopo la rimozione degli outlier, la Random Forest ha migliorato la capacità di generalizzazione mantenendo basso il bias (0.263) e contenendo la varianza. Anche la media F1 dei modelli potati è migliorata. Questo dimostra come l’outlier elimination abbia aiutato a rendere più rappresentativi i dati di training.
3. **Clustering**
La qualità del clustering KMeans si è mantenuta pressoché costante (Silhouette intorno a 0.52–0.58), ma gli altri indici (Calinski-Harabasz e Davies-Bouldin) indicano lievi miglioramenti. DBSCAN continua a fallire nel trovare cluster significativi, suggerendo che il dataset non presenta strutture di densità nette, o che serve una migliore ottimizzazione dei parametri eps/min_samples o una diversa rappresentazione dei dati (feature o metriche).

✅ **Sintesi**
- L’eliminazione degli outlier ha migliorato sensibilmente le prestazioni dei modelli supervisionati, soprattutto nella regressione.
- La Random Forest si conferma un modello stabile e con varianza contenuta, mentre l’errore totale resta principalmente causato da bias, suggerendo la necessità di migliorare le feature.
- La qualità del clustering è discreta ma non ottimale: algoritmi alternativi o una nuova trasformazione delle feature potrebbero migliorare la separabilità.
- DBSCAN non risulta adatto alla struttura del dataset in uso.